In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import h5py

from mpl_toolkits.mplot3d import Axes3D

import sys
sys.path.append("/home/habjan.e/TNG/TNG_cluster_dynamics")
import TNG_DA


dirc_path = '/home/habjan.e/'
sys.path.append(dirc_path + 'TNG/Codes')
import iapi_TNG as iapi

dirc = dirc_path + 'TNG/TNG_workshop/'
sim = 'TNG300-1'

baseUrl = 'http://www.tng-project.org/api/'
r=iapi.get(baseUrl)
TNG_data_path = dirc_path + 'TNG/Data/'

### Pick a cluster index

In [ ]:
cluster_ind = 98

### Import subhalo information

In [ ]:
pos, vel, groups, subhalo_masses, h, halo_mass = TNG_DA.get_cluster_props(cluster_ind)

### Import particle data

In [ ]:
fName = f'/home/habjan.e/TNG/Data/TNG_data/halo_cutouts_dm_{cluster_ind}.hdf5'

with h5py.File(fName, 'r') as f:

    part_coordinates = f['PartType1']['Coordinates'][:]
    velocities = f['PartType1']['Velocities'][:]
    ids = f['PartType1']['ParticleIDs'][:]

masses = np.zeros(part_coordinates.shape[0]) + 5.9 * 10**7

coordinates = TNG_DA.coord_cm_corr(cluster_ind = cluster_ind, coordinates = part_coordinates)

### Import ROCKSTAR pandas df

In [ ]:
output_path = f'/projects/mccleary_group/habjan.e/TNG/Data/rockstar_output/rockstar_subhalos_{cluster_ind}.list'


column_names = ["id", "pos_0", "pos_1", "pos_2", "pos_3", "pos_4", "pos_5", "bulkvel_0", "bulkvel_1", "bulkvel_2", "corevel_0", "corevel_1", "corevel_2", "mass_grav", "p_start", "num_p"]
df = pd.read_csv(output_path, sep=r"\s+", comment="#", names=column_names)
df

### Add member lists to pandas df

In [ ]:
members_path = f'/projects/mccleary_group/habjan.e/TNG/Data/rockstar_output/rockstar_subhalo_members_{cluster_ind}.list'

members = pd.read_csv(members_path,
                      sep=r"\s+", names=["halo_id","particle_id"])

member_id = np.array(members['particle_id'])
mem_halo_id = np.array(members['halo_id'])

grouped = members.groupby("halo_id")["particle_id"].apply(list)
df["member_ids"] = df["id"].map(grouped)
len(df.loc[np.where((df['num_p'] > 10**3) & (df['num_p'] < 10**4))[0]])

### Plot Some subhalos

In [ ]:
fig = plt.figure()
ax  = fig.add_subplot(111, projection='3d')

for i in np.where((df['num_p'] > 10**3) & (df['num_p'] < 10**4))[0]:

    plot_bool = np.isin(ids, df['member_ids'][i])

    ax.scatter(
        coordinates[plot_bool,1],  # x
        coordinates[plot_bool,2],  # y
        coordinates[plot_bool,0],  # z
        s=2              
    )

# label axes
ax.set_xlabel('Y [kpc]')
ax.set_ylabel('Z [kpc]')
ax.set_zlabel('X [kpc]')

plt.tight_layout()
plt.show()

### Make a histogram of the number particles in a substructure

In [ ]:
num_p = np.array(df['num_p'])
num_p_log = np.log10(num_p[num_p > 0])


plt.hist(num_p_log, color='blue', bins = np.linspace(1, 7, 200))

lower_b = np.nanquantile(num_p_log, 0.99)
plt.axvline(lower_b, c='red', linestyle='--')

upper_b = np.nanquantile(num_p_log, 0.9998)
plt.axvline(upper_b, c='green', linestyle='--')

plt.yscale('log')

plt.ylabel('Subhalo Count')
plt.xlabel('TNG particle Count')

### Do a distance calculation to find which subhalo is closest to particles

In [ ]:
#k = int(np.mean(np.array([10**upper_b, 10**lower_b])))
#upper_b = int(10**upper_b)
#lower_b = int(10**lower_b)

k = 5000
upper_b = int(10**4)
lower_b = int(10**3)

print(lower_b, upper_b, k)

sub_mem = np.zeros(pos.shape[0])

real_subs = np.array(df['id'][np.where((df['num_p'] > lower_b) & (df['num_p'] < upper_b))[0]])
real_bool = np.isin(mem_halo_id, real_subs)

for i in range(pos.shape[0]):

    dist = np.sqrt((pos[i, 0] - coordinates[real_bool, 0])**2 + (pos[i, 1] - coordinates[real_bool, 1])**2 + (pos[i, 2] - coordinates[real_bool, 2])**2)

    idx = np.argpartition(dist, k-1)[:k]

    k_ids = ids[real_bool][idx]

    sub_mem[i] = np.unique(mem_halo_id[real_bool][np.isin(member_id[real_bool], k_ids)])[0]

### Plot the found subhalos in their respective substructure

In [ ]:
fig = plt.figure()
ax  = fig.add_subplot(111, projection='3d')

for i in np.unique(sub_mem):

    plot_bool = np.where(i == sub_mem)[0]

    ax.scatter(
        pos[plot_bool,1],  # x
        pos[plot_bool,2],  # y
        pos[plot_bool,0],  # z
        s=100              
    )

# label axes
ax.set_xlabel('Y [kpc]')
ax.set_ylabel('Z [kpc]')
ax.set_zlabel('X [kpc]')

plt.tight_layout()
plt.show()

### The same plot but in velocity space

In [ ]:
fig = plt.figure()
ax  = fig.add_subplot(111, projection='3d')

for i in np.unique(sub_mem):

    plot_bool = np.where(i == sub_mem)[0]

    ax.scatter(
        vel[plot_bool,2],  # x
        vel[plot_bool,0],  # y
        vel[plot_bool,1],  # z
        s=100              
    )

# label axes
ax.set_xlabel(r'$v_{z}$ [$\frac{km}{s}$]')
ax.set_ylabel(r'$v_{x}$ [$\frac{km}{s}$]')
ax.set_zlabel(r'$v_{y}$ [$\frac{km}{s}$]')

plt.tight_layout()
plt.show()

# DS+ comparison to ROCKSTAR

In [ ]:
pos_2d, vel_los = TNG_DA.project_3d_to_2d(pos, vel)
dsp_results, C, P = TNG_DA.run_dsp(pos_2d, vel_los, groups, n_sims=1000, Plim_P = 50, Ng_jump=1)
C, P

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

tng_arr = np.zeros(len(subhalo_masses))
dsp_arr = np.zeros(len(subhalo_masses))

marker_size = 40
gray_size = 15

### TNG subgroups

k = 0

ax1.scatter(0, 0, c='white', alpha=0, label='ROCKSTAR')

for i in np.unique(sub_mem):

    plot_bool = np.where(i == sub_mem)[0]
    
    if len(plot_bool) > 1 and len(plot_bool) < int(np.sqrt(len(subhalo_masses))):
        
        ax1.scatter(pos_2d[plot_bool,0], pos_2d[plot_bool,1], s=marker_size)
        tng_arr[plot_bool] = 1
    
    else:

        ax1.scatter(pos_2d[plot_bool,0], pos_2d[plot_bool,1], s=gray_size, c='gray', alpha =0.4)
        tng_arr[plot_bool] = 2

ax1.legend(loc = 'lower right', fontsize = 12.5)

### DS+ subgroups

sub_grnu, sub_count = np.unique(dsp_results[1][:, 8], return_counts=True)
sub_grnu_arr = dsp_results[1][:, 8]

for i in range(len(sub_grnu)):

    if i == 0:

        ax2.scatter(0, 0, c='white', alpha=0, label='DS+')

    group_dsp_arr = np.where(sub_grnu_arr == sub_grnu[i])[0]
    x = dsp_results[1][group_dsp_arr, 9]
    y = dsp_results[1][group_dsp_arr, 10]

    if sub_count[i] > 1 and sub_count[i] < int(np.sqrt(len(subhalo_masses))):

        ax2.scatter(x, y, s=marker_size)
        dsp_arr[group_dsp_arr] = 1
    
    else:

        ax2.scatter(x, y, s=gray_size, c='gray', alpha =0.4)
        dsp_arr[group_dsp_arr] = 2

ax2.legend(loc = 'lower right', fontsize = 12.5)

### Plot comparing TNG and DS+ substructure

ax3.scatter(0, 0, c='white', alpha=0, label='ROCKSTAR & DS+')

# DS+ correctly identifies a galaxy that is part of substructure
bool_corr = (tng_arr == 1) & (dsp_arr == 1)
ax3.scatter(pos_2d[bool_corr, 0], pos_2d[bool_corr, 1], c = 'green' , s=marker_size)

# Galaxies that are not part of substructure and where not identified by DS +
bool_indiff = (tng_arr == 2) & (dsp_arr == 2)
ax3.scatter(pos_2d[bool_indiff, 0], pos_2d[bool_indiff, 1], s=gray_size, c='gray', alpha =0.4)

# Galaxies that are part of substructure but were not identified by DS+
bool_noid = (tng_arr == 1) & (dsp_arr == 2)
ax3.scatter(pos_2d[bool_noid, 0], pos_2d[bool_noid, 1], c = 'black', s=marker_size)

# Galaxies not part of substructure but were identified by DS+
bool_wrong = (tng_arr == 2) & (dsp_arr == 1)
ax3.scatter(pos_2d[bool_wrong, 0], pos_2d[bool_wrong, 1], c = 'red', s=marker_size)

ax3.legend(loc = 'lower right', fontsize = 12.5)

### Adjust layout and show plot

#fig.suptitle(f'TNG Cluster Index: {cluster_TNG} | Orientation: {view_vector}')
fig.supxlabel(r'X - X$_0$ [kpc]', fontsize = 17.5, y = -0.02)
fig.supylabel(r'Y - Y$_0$ [kpc]', fontsize = 17.5, x = 0.07)

fig.subplots_adjust(wspace=0)

plt.show()

### Import TNG subhalo data that is necessary to do subhalo-based classification

In [ ]:
SubhaloLenType = iapi.getSubhaloField('SubhaloLenType', simulation=sim, snapshot=99, fileName=TNG_data_path+'TNG_data/'+sim+'_SubhaloLenType', rewriteFile=0)
Group_num = iapi.getSubhaloField('SubhaloGrNr', simulation=sim, fileName=TNG_data_path+'TNG_data/'+sim+'_SubhaloGrNr', rewriteFile=0)
SubhaloStellarPhotometrics = iapi.getSubhaloField('SubhaloStellarPhotometrics', snapshot=99, simulation=sim, fileName=TNG_data_path+'TNG_data/'+sim+'_SubhaloStellarPhotometrics', rewriteFile=0)

sub_ind = np.where(Group_num == cluster_ind)[0]

sub_lens, sub_photo = SubhaloLenType[sub_ind, :], SubhaloStellarPhotometrics[sub_ind, :]

mag_cut = -18
bright_ind = sub_photo[:, 4] < mag_cut

sub_lens_type = sub_lens[bright_ind, :]

### Compare the number of particles in TNG subhalos with ROCKSTAR subhalos

In [ ]:
num_p = np.array(df['num_p'])
num_p_log = np.log10(num_p[num_p > 0])

sub_lens_log = np.log10(sub_lens[sub_lens > 0])

plt.hist(num_p_log, color='blue', bins = np.linspace(1, 7, 200), label = 'ROCKSTAR')
plt.hist(sub_lens_log, color='red', bins = np.linspace(1, 7, 200), label = 'SUBFIND')

lower_b = np.nanquantile(num_p_log, 0.99)
#plt.axvline(lower_b, c='red', linestyle='--')

upper_b = np.nanquantile(num_p_log, 0.9998)
#plt.axvline(upper_b, c='green', linestyle='--')

plt.yscale('log')

plt.ylabel('Subhalo Count')
plt.xlabel('TNG particle Count')

plt.legend(loc='upper right')

### Code to sort subhalos into a substructure

In [ ]:
sub_mem_len = np.zeros(pos.shape[0])

part_count = 0
bright_sub = 0

for i in range(sub_lens[:, 1].shape[0]):

    sub_parts_i = sub_lens[i, 1]

    if sub_photo[i, 4] < mag_cut:

        sub_parts = ids[int(part_count) : int(part_count + sub_parts_i)]
        part_count += sub_parts_i

        sub_bool = np.isin(member_id, sub_parts)
        sub_members = mem_halo_id[sub_bool]

        try: 

            sub_mem_len[bright_sub] = np.unique(sub_members)[0]
        
        except: 

            sub_mem_len[bright_sub] = np.nan. ### This exception is solely for subhalos with no particles associated with it

        bright_sub += 1

        if bright_sub + 1 == pos.shape[0]:
            break

    else:

        part_count += sub_parts_i

        continue


### Print the TNG-based substructure results

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

tng_arr = np.zeros(len(subhalo_masses))
dsp_arr = np.zeros(len(subhalo_masses))

marker_size = 40
gray_size = 15

### TNG subgroups

k = 0

ax1.scatter(0, 0, c='white', alpha=0, label='ROCKSTAR')

for i in np.unique(sub_mem_len):

    plot_bool = np.where(i == sub_mem_len)[0]
    
    if len(plot_bool) > 1 and len(plot_bool) < int(np.sqrt(len(subhalo_masses))):
        
        ax1.scatter(pos_2d[plot_bool,0], pos_2d[plot_bool,1], s=marker_size)
        tng_arr[plot_bool] = 1
        ax1.scatter(df['pos_0'].loc[i], df['pos_1'].loc[i], s = df['num_p'].loc[i])
    
    else:

        ax1.scatter(pos_2d[plot_bool,0], pos_2d[plot_bool,1], s=gray_size, c='gray', alpha =0.4)
        tng_arr[plot_bool] = 2

ax1.legend(loc = 'lower right', fontsize = 12.5)

### DS+ subgroups

sub_grnu, sub_count = np.unique(dsp_results[1][:, 8], return_counts=True)
sub_grnu_arr = dsp_results[1][:, 8]

for i in range(len(sub_grnu)):

    if i == 0:

        ax2.scatter(0, 0, c='white', alpha=0, label='DS+')

    group_dsp_arr = np.where(sub_grnu_arr == sub_grnu[i])[0]
    x = dsp_results[1][group_dsp_arr, 9]
    y = dsp_results[1][group_dsp_arr, 10]

    if sub_count[i] > 1 and sub_count[i] < int(np.sqrt(len(subhalo_masses))):

        ax2.scatter(x, y, s=marker_size)
        dsp_arr[group_dsp_arr] = 1
    
    else:

        ax2.scatter(x, y, s=gray_size, c='gray', alpha =0.4)
        dsp_arr[group_dsp_arr] = 2

ax2.legend(loc = 'lower right', fontsize = 12.5)

### Plot comparing TNG and DS+ substructure

ax3.scatter(0, 0, c='white', alpha=0, label='ROCKSTAR & DS+')

# DS+ correctly identifies a galaxy that is part of substructure
bool_corr = (tng_arr == 1) & (dsp_arr == 1)
ax3.scatter(pos_2d[bool_corr, 0], pos_2d[bool_corr, 1], c = 'green' , s=marker_size)

# Galaxies that are not part of substructure and where not identified by DS +
bool_indiff = (tng_arr == 2) & (dsp_arr == 2)
ax3.scatter(pos_2d[bool_indiff, 0], pos_2d[bool_indiff, 1], s=gray_size, c='gray', alpha =0.4)

# Galaxies that are part of substructure but were not identified by DS+
bool_noid = (tng_arr == 1) & (dsp_arr == 2)
ax3.scatter(pos_2d[bool_noid, 0], pos_2d[bool_noid, 1], c = 'black', s=marker_size)

# Galaxies not part of substructure but were identified by DS+
bool_wrong = (tng_arr == 2) & (dsp_arr == 1)
ax3.scatter(pos_2d[bool_wrong, 0], pos_2d[bool_wrong, 1], c = 'red', s=marker_size)

ax3.legend(loc = 'lower right', fontsize = 12.5)

### Adjust layout and show plot

#fig.suptitle(f'TNG Cluster Index: {cluster_TNG} | Orientation: {view_vector}')
fig.supxlabel(r'X - X$_0$ [kpc]', fontsize = 17.5, y = -0.02)
fig.supylabel(r'Y - Y$_0$ [kpc]', fontsize = 17.5, x = 0.07)

fig.subplots_adjust(wspace=0)

plt.show()